# 💾 Carga de Dados - Segurança Pública SP

Este notebook realiza a **carga** dos dados processados em destinos finais (arquivos consolidados e bancos de dados).

## Objetivos:
- Carregar dados processados de `data/processed/`
- Consolidar dados de múltiplos anos em um único arquivo
- Salvar em formato Parquet otimizado
- Carregar em banco de dados (DuckDB/SQLite)
- Criar relatórios de resumo
- Validar integridade dos dados carregados

---

## 1. Importar Bibliotecas e Funções de Carga

In [ ]:
# Importar bibliotecas necessárias
import sys
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import logging
import json

# Adicionar o diretório src ao path
sys.path.append(str(Path().resolve().parent / 'src'))

# Importar funções de carga
from load import (
    save_to_csv,
    save_to_parquet,
    save_to_excel,
    save_to_database,
    save_metadata,
    create_summary_report
)

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Bibliotecas e funções de carga importadas com sucesso!")

## 2. Configurar Diretórios e Parâmetros

In [ ]:
# Definir diretórios do projeto
PROJECT_ROOT = Path().resolve().parent
DATA_PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
DATA_FINAL_DIR = PROJECT_ROOT / 'data' / 'final'
DATABASE_DIR = PROJECT_ROOT / 'data' / 'database'

# Criar diretórios se não existirem
DATA_FINAL_DIR.mkdir(parents=True, exist_ok=True)
DATABASE_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Diretório de dados processados: {DATA_PROCESSED_DIR}")
print(f"📁 Diretório de dados finais: {DATA_FINAL_DIR}")
print(f"📁 Diretório de banco de dados: {DATABASE_DIR}")

## 3. Carregar Dados Processados

In [ ]:
# Listar arquivos processados disponíveis
print("📂 Arquivos disponíveis em data/processed/:\n")

csv_files = list(DATA_PROCESSED_DIR.glob('dados_processados_*.csv'))
parquet_files = list(DATA_PROCESSED_DIR.glob('dados_processados_*.parquet'))

print(f"📊 Arquivos CSV: {len(csv_files)}")
for file in csv_files:
    size_kb = file.stat().st_size / 1024
    print(f"  - {file.name} ({size_kb:.2f} KB)")

print(f"\n📊 Arquivos Parquet: {len(parquet_files)}")
for file in parquet_files:
    size_kb = file.stat().st_size / 1024
    print(f"  - {file.name} ({size_kb:.2f} KB)")

# Carregar dados processados (preferir Parquet se disponível)
if parquet_files:
    arquivo_carga = parquet_files[0]
    df_processed = pd.read_parquet(arquivo_carga)
    print(f"\n✅ Dados carregados de: {arquivo_carga.name}")
elif csv_files:
    arquivo_carga = csv_files[0]
    df_processed = pd.read_csv(arquivo_carga, encoding='utf-8-sig')
    print(f"\n✅ Dados carregados de: {arquivo_carga.name}")
else:
    print("\n⚠️ Nenhum arquivo processado encontrado!")
    print("Execute primeiro o notebook '2_transformacao.ipynb'")
    df_processed = None

if df_processed is not None:
    print(f"📊 Dimensões: {df_processed.shape[0]:,} linhas x {df_processed.shape[1]} colunas")
    print(f"\n📋 Primeiras linhas:")
    display(df_processed.head())

## 4. Consolidar Dados de Múltiplos Anos

Simulando a consolidação de dados de diferentes anos/períodos:

In [ ]:
def consolidar_multiplos_anos(data_dir: Path, pattern: str = 'dados_processados_*.parquet'):
    """
    Consolida arquivos de múltiplos anos em um único DataFrame
    
    Args:
        data_dir: Diretório contendo os arquivos
        pattern: Padrão de nome dos arquivos a consolidar
    
    Returns:
        DataFrame consolidado
    """
    arquivos = list(data_dir.glob(pattern))
    
    if not arquivos:
        # Se não encontrar Parquet, tentar CSV
        pattern = pattern.replace('.parquet', '.csv')
        arquivos = list(data_dir.glob(pattern))
    
    if not arquivos:
        logger.warning(f"Nenhum arquivo encontrado com o padrão: {pattern}")
        return None
    
    print(f"📂 Consolidando {len(arquivos)} arquivo(s)...\n")
    
    dataframes = []
    for arquivo in arquivos:
        print(f"  📄 Carregando: {arquivo.name}")
        
        if arquivo.suffix == '.parquet':
            df = pd.read_parquet(arquivo)
        else:
            df = pd.read_csv(arquivo, encoding='utf-8-sig')
        
        dataframes.append(df)
        print(f"     → {len(df):,} registros")
    
    # Concatenar todos os DataFrames
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    
    print(f"\n✅ Consolidação concluída!")
    print(f"📊 Total de registros consolidados: {len(df_consolidado):,}")
    
    return df_consolidado

# Executar consolidação
df_consolidado = consolidar_multiplos_anos(DATA_PROCESSED_DIR)

if df_consolidado is not None:
    # Remover duplicatas que possam existir após consolidação
    registros_antes = len(df_consolidado)
    df_consolidado = df_consolidado.drop_duplicates()
    registros_depois = len(df_consolidado)
    
    if registros_antes > registros_depois:
        print(f"\n🗑️  Removidas {registros_antes - registros_depois:,} duplicatas após consolidação")
    
    print(f"\n📋 Estrutura dos dados consolidados:")
    print(df_consolidado.info())

## 5. Salvar em Arquivo Parquet Consolidado

Usando a função `save_to_parquet()` para salvar o arquivo final otimizado:

In [ ]:
if df_consolidado is not None:
    # Nome do arquivo consolidado
    arquivo_final = DATA_FINAL_DIR / 'seguranca_publica_sp_consolidado.parquet'
    
    # Salvar usando a função save_to_parquet
    print("💾 Salvando arquivo Parquet consolidado...")
    sucesso = save_to_parquet(
        df_consolidado, 
        str(arquivo_final),
        compression='snappy'  # Compressão eficiente
    )
    
    if sucesso:
        # Verificar tamanho do arquivo
        size_mb = arquivo_final.stat().st_size / (1024 * 1024)
        print(f"\n✅ Arquivo Parquet salvo com sucesso!")
        print(f"📁 Caminho: {arquivo_final}")
        print(f"💾 Tamanho: {size_mb:.2f} MB")
        print(f"📊 Registros: {len(df_consolidado):,}")
        print(f"📋 Colunas: {len(df_consolidado.columns)}")
        
        # Comparar com CSV (para demonstrar eficiência do Parquet)
        arquivo_csv_temp = DATA_FINAL_DIR / 'temp_comparison.csv'
        df_consolidado.to_csv(arquivo_csv_temp, index=False, encoding='utf-8-sig')
        size_csv_mb = arquivo_csv_temp.stat().st_size / (1024 * 1024)
        
        print(f"\n📊 Comparação de Tamanho:")
        print(f"  CSV:     {size_csv_mb:.2f} MB")
        print(f"  Parquet: {size_mb:.2f} MB")
        print(f"  Economia: {((size_csv_mb - size_mb) / size_csv_mb * 100):.1f}%")
        
        # Remover arquivo temporário
        arquivo_csv_temp.unlink()
else:
    print("⚠️ Nenhum dado para salvar.")

## 6. Carregar em Banco de Dados SQLite

Usando a função `save_to_database()` para criar um banco SQLite:

In [ ]:
if df_consolidado is not None:
    # Configurar conexão SQLite
    db_path = DATABASE_DIR / 'seguranca_publica_sp.db'
    connection_string = f'sqlite:///{db_path}'
    
    print("🗄️  Carregando dados no SQLite...")
    print(f"📁 Banco de dados: {db_path}\n")
    
    # Salvar tabela principal
    sucesso = save_to_database(
        df_consolidado,
        table_name='ocorrencias_criminais',
        connection_string=connection_string,
        if_exists='replace'  # Substituir se já existir
    )
    
    if sucesso:
        print(f"\n✅ Dados carregados na tabela 'ocorrencias_criminais'")
        print(f"💾 Tamanho do banco: {db_path.stat().st_size / (1024 * 1024):.2f} MB")
        
        # Criar tabelas agregadas adicionais
        print("\n📊 Criando tabelas agregadas...")
        
        # Tabela agregada por município
        df_por_municipio = df_consolidado.groupby('municipio').agg({
            'ocorrencias': 'sum',
            'vitimas': 'sum',
            'taxa_criminalidade': 'mean'
        }).reset_index()
        
        save_to_database(
            df_por_municipio,
            table_name='agregado_por_municipio',
            connection_string=connection_string,
            if_exists='replace'
        )
        
        # Tabela agregada por tipo de crime
        df_por_crime = df_consolidado.groupby('tipo_crime').agg({
            'ocorrencias': 'sum',
            'vitimas': 'sum'
        }).reset_index().sort_values('ocorrencias', ascending=False)
        
        save_to_database(
            df_por_crime,
            table_name='agregado_por_tipo_crime',
            connection_string=connection_string,
            if_exists='replace'
        )
        
        print("\n✅ Tabelas agregadas criadas com sucesso!")
        print("   - ocorrencias_criminais (principal)")
        print("   - agregado_por_municipio")
        print("   - agregado_por_tipo_crime")
else:
    print("⚠️ Nenhum dado para carregar.")

## 7. Carregar em DuckDB (Alternativa Otimizada)

DuckDB é mais eficiente para análises de dados em larga escala:

In [ ]:
try:
    import duckdb
    
    if df_consolidado is not None:
        # Criar banco DuckDB
        duckdb_path = DATABASE_DIR / 'seguranca_publica_sp.duckdb'
        
        print("🦆 Carregando dados no DuckDB...")
        print(f"📁 Banco de dados: {duckdb_path}\n")
        
        # Conectar ao DuckDB
        conn = duckdb.connect(str(duckdb_path))
        
        # Criar tabela a partir do DataFrame
        conn.execute("DROP TABLE IF EXISTS ocorrencias_criminais")
        conn.execute("CREATE TABLE ocorrencias_criminais AS SELECT * FROM df_consolidado")
        
        # Verificar criação
        total_registros = conn.execute("SELECT COUNT(*) FROM ocorrencias_criminais").fetchone()[0]
        print(f"✅ Tabela 'ocorrencias_criminais' criada com {total_registros:,} registros")
        
        # Criar índices para melhor performance
        print("\n📑 Criando índices...")
        conn.execute("CREATE INDEX idx_municipio ON ocorrencias_criminais(municipio)")
        conn.execute("CREATE INDEX idx_tipo_crime ON ocorrencias_criminais(tipo_crime)")
        conn.execute("CREATE INDEX idx_ano_mes ON ocorrencias_criminais(ano, mes)")
        print("✅ Índices criados")
        
        # Exemplo de query analítica rápida
        print("\n📊 Teste de Performance - Top 5 Municípios:")
        resultado = conn.execute("""
            SELECT 
                municipio,
                SUM(ocorrencias) as total_ocorrencias,
                ROUND(AVG(taxa_criminalidade), 2) as taxa_media
            FROM ocorrencias_criminais
            GROUP BY municipio
            ORDER BY total_ocorrencias DESC
            LIMIT 5
        """).fetchdf()
        
        print(resultado)
        
        # Tamanho do banco
        print(f"\n💾 Tamanho do banco DuckDB: {duckdb_path.stat().st_size / (1024 * 1024):.2f} MB")
        
        conn.close()
        print("\n✅ DuckDB configurado com sucesso!")
        
except ImportError:
    print("⚠️ DuckDB não está instalado.")
    print("Para instalar: pip install duckdb")
except Exception as e:
    print(f"❌ Erro ao configurar DuckDB: {e}")

## 8. Validar Dados Carregados

Verificar integridade dos dados após a carga:

In [ ]:
if df_consolidado is not None:
    print("="*60)
    print("🔍 VALIDAÇÃO DOS DADOS CARREGADOS")
    print("="*60)
    
    # 1. Verificar arquivo Parquet
    arquivo_parquet = DATA_FINAL_DIR / 'seguranca_publica_sp_consolidado.parquet'
    if arquivo_parquet.exists():
        df_parquet_test = pd.read_parquet(arquivo_parquet)
        print(f"\n✅ Arquivo Parquet:")
        print(f"   📊 Registros: {len(df_parquet_test):,}")
        print(f"   📋 Colunas: {len(df_parquet_test.columns)}")
        print(f"   💾 Tamanho: {arquivo_parquet.stat().st_size / (1024*1024):.2f} MB")
    
    # 2. Verificar banco SQLite
    db_sqlite = DATABASE_DIR / 'seguranca_publica_sp.db'
    if db_sqlite.exists():
        from sqlalchemy import create_engine
        engine = create_engine(f'sqlite:///{db_sqlite}')
        
        # Contar registros
        query = "SELECT COUNT(*) as total FROM ocorrencias_criminais"
        resultado = pd.read_sql(query, engine)
        
        print(f"\n✅ Banco SQLite:")
        print(f"   📊 Registros: {resultado['total'].iloc[0]:,}")
        print(f"   💾 Tamanho: {db_sqlite.stat().st_size / (1024*1024):.2f} MB")
        
        # Verificar tabelas
        query_tabelas = "SELECT name FROM sqlite_master WHERE type='table'"
        tabelas = pd.read_sql(query_tabelas, engine)
        print(f"   📑 Tabelas: {', '.join(tabelas['name'].tolist())}")
    
    # 3. Verificar banco DuckDB
    db_duckdb = DATABASE_DIR / 'seguranca_publica_sp.duckdb'
    if db_duckdb.exists():
        try:
            import duckdb
            conn = duckdb.connect(str(db_duckdb))
            total = conn.execute("SELECT COUNT(*) FROM ocorrencias_criminais").fetchone()[0]
            
            print(f"\n✅ Banco DuckDB:")
            print(f"   📊 Registros: {total:,}")
            print(f"   💾 Tamanho: {db_duckdb.stat().st_size / (1024*1024):.2f} MB")
            
            conn.close()
        except:
            pass
    
    # 4. Comparação de integridade
    print(f"\n📊 Comparação de Integridade:")
    print(f"   DataFrame Original: {len(df_consolidado):,} registros")
    if arquivo_parquet.exists():
        print(f"   Arquivo Parquet:    {len(df_parquet_test):,} registros")
        if len(df_consolidado) == len(df_parquet_test):
            print("   ✅ Integridade OK - Parquet")
    
    print("\n" + "="*60)
else:
    print("⚠️ Nenhum dado para validar.")

## 9. Criar Relatório de Resumo

Usando a função `create_summary_report()` para documentar a carga:

In [ ]:
if df_consolidado is not None:
    # Adicionar coluna 'data' para o relatório (se não existir)
    if 'data' not in df_consolidado.columns and 'ano' in df_consolidado.columns and 'mes' in df_consolidado.columns:
        df_consolidado['data'] = pd.to_datetime(
            df_consolidado['ano'].astype(str) + '-' + 
            df_consolidado['mes'].astype(str).str.zfill(2) + '-01'
        )
    
    # Criar relatório usando a função
    print("📝 Gerando relatório de resumo...")
    sucesso = create_summary_report(df_consolidado, str(DATA_FINAL_DIR))
    
    if sucesso:
        # Ler e exibir o relatório
        summary_path = DATA_FINAL_DIR / 'summary_report.json'
        with open(summary_path, 'r', encoding='utf-8') as f:
            summary = json.load(f)
        
        print("\n✅ Relatório de resumo criado!")
        print(f"📁 Caminho: {summary_path}")
        print("\n📋 Conteúdo do Relatório:")
        print(json.dumps(summary, indent=2, ensure_ascii=False))
else:
    print("⚠️ Nenhum dado para gerar relatório.")

## 10. Salvar Metadados da Carga

Usando a função `save_metadata()` para documentar o processo:

In [ ]:
if df_consolidado is not None:
    # Criar metadados da carga
    metadata_carga = {
        'data_carga': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'notebook': '3_carga.ipynb',
        'etapa': 'CARGA - Consolidação e Armazenamento',
        'dados_consolidados': {
            'total_registros': int(len(df_consolidado)),
            'total_colunas': int(len(df_consolidado.columns)),
            'colunas': list(df_consolidado.columns),
            'periodo': {
                'ano_inicio': int(df_consolidado['ano'].min()),
                'ano_fim': int(df_consolidado['ano'].max()),
                'mes_inicio': int(df_consolidado['mes'].min()),
                'mes_fim': int(df_consolidado['mes'].max())
            }
        },
        'arquivos_gerados': {
            'parquet': 'seguranca_publica_sp_consolidado.parquet',
            'banco_sqlite': 'seguranca_publica_sp.db',
            'banco_duckdb': 'seguranca_publica_sp.duckdb'
        },
        'destinos': [
            'Arquivo Parquet consolidado (data/final/)',
            'Banco de dados SQLite (data/database/)',
            'Banco de dados DuckDB (data/database/)'
        ],
        'tabelas_criadas': {
            'sqlite': [
                'ocorrencias_criminais',
                'agregado_por_municipio',
                'agregado_por_tipo_crime'
            ],
            'duckdb': [
                'ocorrencias_criminais'
            ]
        },
        'estatisticas': {
            'total_ocorrencias': int(df_consolidado['ocorrencias'].sum()),
            'total_vitimas': int(df_consolidado['vitimas'].sum()),
            'taxa_media_criminalidade': float(df_consolidado['taxa_criminalidade'].mean()),
            'municipios': int(df_consolidado['municipio'].nunique()),
            'tipos_crime': int(df_consolidado['tipo_crime'].nunique())
        },
        'qualidade': {
            'valores_nulos': int(df_consolidado.isnull().sum().sum()),
            'duplicatas': int(df_consolidado.duplicated().sum()),
            'integridade': 'OK'
        }
    }
    
    # Salvar metadados usando a função
    metadata_path = DATA_FINAL_DIR / 'metadata_carga.json'
    sucesso = save_metadata(metadata_carga, str(metadata_path))
    
    if sucesso:
        print("✅ Metadados da carga salvos com sucesso!")
        print(f"📁 Caminho: {metadata_path}")
        print(f"\n📋 Resumo dos Metadados:")
        print(json.dumps(metadata_carga, indent=2, ensure_ascii=False))
else:
    print("⚠️ Nenhum dado para gerar metadados.")

## 11. Exemplos de Consultas nos Bancos de Dados

In [ ]:
# Exemplo 1: Consulta no SQLite
db_sqlite = DATABASE_DIR / 'seguranca_publica_sp.db'

if db_sqlite.exists():
    from sqlalchemy import create_engine
    engine = create_engine(f'sqlite:///{db_sqlite}')
    
    print("="*60)
    print("🔍 EXEMPLOS DE CONSULTAS SQL - SQLite")
    print("="*60)
    
    # Query 1: Top 10 municípios com mais ocorrências
    print("\n1️⃣ Top 10 Municípios com Mais Ocorrências:")
    query1 = """
        SELECT 
            municipio,
            SUM(ocorrencias) as total_ocorrencias,
            ROUND(AVG(taxa_criminalidade), 2) as taxa_media
        FROM ocorrencias_criminais
        GROUP BY municipio
        ORDER BY total_ocorrencias DESC
        LIMIT 10
    """
    resultado1 = pd.read_sql(query1, engine)
    print(resultado1.to_string(index=False))
    
    # Query 2: Evolução mensal de crimes
    print("\n\n2️⃣ Evolução Mensal de Crimes:")
    query2 = """
        SELECT 
            ano,
            mes,
            SUM(ocorrencias) as total_ocorrencias
        FROM ocorrencias_criminais
        GROUP BY ano, mes
        ORDER BY ano, mes
    """
    resultado2 = pd.read_sql(query2, engine)
    print(resultado2.to_string(index=False))
    
    # Query 3: Crimes por categoria
    print("\n\n3️⃣ Total de Ocorrências por Categoria:")
    query3 = """
        SELECT 
            categoria_crime,
            COUNT(*) as quantidade_registros,
            SUM(ocorrencias) as total_ocorrencias,
            SUM(vitimas) as total_vitimas
        FROM ocorrencias_criminais
        GROUP BY categoria_crime
        ORDER BY total_ocorrencias DESC
    """
    resultado3 = pd.read_sql(query3, engine)
    print(resultado3.to_string(index=False))
    
    print("\n" + "="*60)
else:
    print("⚠️ Banco SQLite não encontrado.")

In [ ]:
# Exemplo 2: Consulta no DuckDB (mais rápido para análises)
db_duckdb = DATABASE_DIR / 'seguranca_publica_sp.duckdb'

if db_duckdb.exists():
    try:
        import duckdb
        
        print("="*60)
        print("🦆 EXEMPLOS DE CONSULTAS SQL - DuckDB")
        print("="*60)
        
        conn = duckdb.connect(str(db_duckdb))
        
        # Query 1: Análise agregada complexa
        print("\n1️⃣ Análise Comparativa por Município (Top 5):")
        query_duckdb = """
            SELECT 
                municipio,
                COUNT(DISTINCT tipo_crime) as tipos_crime_distintos,
                SUM(ocorrencias) as total_ocorrencias,
                ROUND(AVG(taxa_criminalidade), 2) as taxa_media,
                MAX(ocorrencias) as max_ocorrencias_mes
            FROM ocorrencias_criminais
            GROUP BY municipio
            ORDER BY total_ocorrencias DESC
            LIMIT 5
        """
        resultado_duckdb = conn.execute(query_duckdb).fetchdf()
        print(resultado_duckdb.to_string(index=False))
        
        # Query 2: Tendência temporal
        print("\n\n2️⃣ Tendência de Crimes Violentos ao Longo do Tempo:")
        query_tendencia = """
            SELECT 
                ano,
                mes,
                categoria_crime,
                SUM(ocorrencias) as total
            FROM ocorrencias_criminais
            WHERE categoria_crime = 'Crimes Violentos'
            GROUP BY ano, mes, categoria_crime
            ORDER BY ano, mes
        """
        resultado_tendencia = conn.execute(query_tendencia).fetchdf()
        print(resultado_tendencia.to_string(index=False))
        
        conn.close()
        print("\n" + "="*60)
        
    except ImportError:
        print("⚠️ DuckDB não está instalado.")
    except Exception as e:
        print(f"❌ Erro: {e}")
else:
    print("⚠️ Banco DuckDB não encontrado.")

## 12. Resumo Final e Próximos Passos

In [ ]:
# Resumo completo do processo de carga
print("="*70)
print("📊 RESUMO COMPLETO DO PROCESSO DE CARGA")
print("="*70)

print("\n✅ ETAPAS CONCLUÍDAS:")
print("  1. ✓ Carregamento de dados processados")
print("  2. ✓ Consolidação de múltiplos anos/períodos")
print("  3. ✓ Salvamento em arquivo Parquet otimizado")
print("  4. ✓ Carga em banco de dados SQLite")
print("  5. ✓ Carga em banco de dados DuckDB")
print("  6. ✓ Validação de integridade dos dados")
print("  7. ✓ Criação de relatórios e metadados")
print("  8. ✓ Exemplos de consultas SQL")

print("\n📁 ARQUIVOS E BANCOS CRIADOS:")

# Listar arquivos em data/final
print("\n  📂 data/final/:")
if DATA_FINAL_DIR.exists():
    for file in sorted(DATA_FINAL_DIR.glob('*')):
        size = file.stat().st_size
        if size > 1024*1024:
            size_str = f"{size/(1024*1024):.2f} MB"
        else:
            size_str = f"{size/1024:.2f} KB"
        print(f"    📄 {file.name:<45} ({size_str})")

# Listar bancos em data/database
print("\n  📂 data/database/:")
if DATABASE_DIR.exists():
    for file in sorted(DATABASE_DIR.glob('*')):
        size = file.stat().st_size
        if size > 1024*1024:
            size_str = f"{size/(1024*1024):.2f} MB"
        else:
            size_str = f"{size/1024:.2f} KB"
        print(f"    🗄️  {file.name:<45} ({size_str})")

print("\n" + "="*70)
print("✅ PROCESSO DE CARGA CONCLUÍDO COM SUCESSO!")
print("="*70)

print("\n🎯 DADOS PRONTOS PARA:")
print("  • Análises exploratórias avançadas")
print("  • Criação de dashboards e visualizações")
print("  • Machine Learning e modelagem preditiva")
print("  • Integração com ferramentas de BI (Power BI, Tableau, etc.)")
print("  • APIs e aplicações web")

print("\n📚 COMO ACESSAR OS DADOS:")
print("  • Parquet: pd.read_parquet('data/final/seguranca_publica_sp_consolidado.parquet')")
print("  • SQLite:  pd.read_sql('SELECT * FROM ocorrencias_criminais', engine)")
print("  • DuckDB:  conn.execute('SELECT * FROM ocorrencias_criminais').fetchdf()")

print("\n🚀 PRÓXIMO PASSO:")
print("  Execute o notebook '4_analise_dados.ipynb' para análises exploratórias!")

print("\n" + "="*70)